# Сравнение двух произвольных структур в формате JSON

In [2]:
#Вспомогательная функция. Соединяет два пути.
#Если первый путь не пустой, то добавляет между путями точку.
def join_paths(path1, path2):
    if len(path1) == 0:
        return path2
    return path1 + "." + path2

#Вспомогательная функция.
#Проверка типа значения элемента списка или ключа словаря.
def is_supported_type(value_type):
    if (value_type == bool or value_type == int or value_type == float or 
        value_type == str or value_type == list or value_type == dict or
        value_type == type(None)):
        return True
    return False

def compare_lists(list1, list2, path, result):
    len1 = len(list1)
    len2 = len(list2)

    if (len1 != len2):
        result += ("Несовпадение количества элементов в списках \"" + path + 
                   "\" (" + str(len1) + " и " + str(len2) + " элемент(-ов))\n")
    
    min_len = min(len1, len2)

    for i in range(min_len):
        t1 = type(list1[i])
        if not is_supported_type(t1):
            result += "ОШИБКА: Некорректный синтаксис первой JSON-структуры: элемент \"" + path + "[" + str(i) + "] имеет тип " + str(t1) + "\n"
            continue

        t2 = type(list2[i])
        if not is_supported_type(t2):
            result += "ОШИБКА: Некорректный синтаксис второй JSON-структуры: элемент \"" + path + "[" + str(i) + "] имеет тип " + str(t2) + "\n"
            continue

        if t1 == t2:
            if (t1 == bool or t1 == float or t1 == int or 
                t1 == str or t1 == type(None)):
                if (list1[i] != list2[i]):
                    result += "Несовпадение значений элементов \"" + path + "[" + str(i) + "]\"\n"
            elif t1 == list:
                result = compare_lists(list1[i], list2[i], path + "[" + str(i) + "]", result)
            else:
                result = compare_dicts(list1[i], list2[i], path + "[" + str(i) + "]", result)
        else:
            result += "Несовпадение типов значений элементов \"" + path + "[" + str(i) + "]\": " + str(t1) + " и " + str(t2) + "\n"

    max_len = max(len1, len2)
    longest_list = list1 if min_len == len2 else list2
    struct_num = "первой" if min_len == len2 else "второй"

    for i in range(max_len - min_len):
        t = type(longest_list[i + min_len])
        if not is_supported_type(t):
            result += "ОШИБКА: Некорректный синтаксис " + struct_num + " JSON-структуры: элемент \"" + path + "[" + str(i + min_len) + "]\" имеет тип " + str(t) + "\n"

    return result

def compare_dicts(dict1, dict2, path, result):
    common_keys = set(dict1.keys()).intersection(set(dict2.keys()))
    dict1_specific_keys = set(dict1.keys()).difference(common_keys)
    dict2_specific_keys = set(dict2.keys()).difference(common_keys)

    for i in dict1_specific_keys:
        if not is_supported_type(type(dict1[i])):
            result += ("ОШИБКА: Некорректный синтаксис первой JSON-структуры: ключ \"" + 
                       join_paths(path, i) + "\" имеет тип " + str(t1) + "\n")
        result += "Ключ \"" + join_paths(path, str(i)) + "\" отсутствует во второй JSON-структуре\n"

    for i in dict2_specific_keys:
        if not is_supported_type(type(dict2[i])):
            result += ("ОШИБКА: Некорректный синтаксис второй JSON-структуры: ключ \"" + 
                       join_paths(path, i) + "\" имеет тип " + str(t2) + "\n")
        result += "Ключ \"" + join_paths(path, str(i)) + "\" отсутствует в первой JSON-структуре\n"

    for i in common_keys:
        t1 = type(dict1[i])
        if not is_supported_type(t1):
            result += ("ОШИБКА: Некорректный синтаксис первой JSON-структуры: ключ \"" + 
                       join_paths(path, i) + "\" имеет тип " + str(t1) + "\n")
            continue

        t2 = type(dict2[i])
        if not is_supported_type(t2):
            result += ("ОШИБКА: Некорректный синтаксис второй JSON-структуры: ключ \"" + 
                       join_paths(path, i) + "\" имеет тип " + str(t2) + "\n")
            continue

        if t1 == t2:
            if (t1 == bool or t1 == float or t1 == int or 
                t1 == str or dict1[i] == None):
                if (dict1[i] != dict2[i]):
                    result += "Несовпадение значений ключа \"" + join_paths(path, str(i)) + "\"\n"
            elif t1 == list:
                result = compare_lists(dict1[i], dict2[i], join_paths(path, str(i)), result)
            else:
                result = compare_dicts(dict1[i], dict2[i], join_paths(path, str(i)), result)
        else:
            result += "Несовпадение типов значений ключа \"" + join_paths(path, i) + "\": " + str(t1) + " и " + str(t2) + " \n"

    return result

#Данная функция выполняет сравнение произвольной структуры json1 с произвольной структурой json2 в формате JSON.
#Присутствует поддержка "None" в качестве ключа словаря или значения. При встрече синтаксической ошибки  
#элемент пропускается, после чего к выводу добавляется соответствующее сообщение.
#Возвращаемое значение: (<Наличие разницы (логическое значение)>, <Разница (строка)>)
def compare_json(json1, json2):
    if ((type(json1) != list and type(json1) != dict) or
        (type(json2) != list and type(json2) != dict)):
        return (False, "ОШИБКА: Входные данные должны быть словарями или списками") 
    
    if type(json1) != type(json2):
        return (False, "Полное несовпадение JSON-структур")
    
    result = ""

    if type(json1) == list:
        result = compare_lists(json1, json2, "", result)
    else:
        result = compare_dicts(json1, json2, "", result)
    
    return (len(result) == 0, result)

In [ ]:
#Загрузка сериализованного JSON из файлов.

import pickle

f = open("gemma_extracted_info.bin", "rb")
extracted_info = pickle.loads(f.read())
f.close()
print(extracted_info)

In [ ]:
#Пример сравнения JSON-структур.
result = compare_json({"key":1,"key2":2,"key3":3,"common_key":[1, 2, 3, (), (), ()]}, {"common_key":[1, 2, 3], None:None})

print("Совпадение: " + str(result[0]))
if not result[0]:
    print()
    print(result[1])